# Projekt PSI
Analiza 5: co jest silniejsze w niespójnym warunku (inc): dźwięk vs obraz? a może negatywne vs pozytywne? (np. nieważne czy negatywny jest dźwięk czy obraz, ważne że co najmniej jedno jest negatywne i całość też jest negatywna)


## Loading data

In [4]:
%matplotlib inline
import pandas

In [10]:

images_params = pandas.read_csv("../data/IAPS.csv", sep=";", header=0, decimal=",")
sounds_params = pandas.read_csv("../data/IADS2.csv", sep=";", header=0, decimal=",")



            Sound  Number ValenceMean ValenceSD ArousalMean ArousalSD
0             Cat     102        4.63      2.17        4.91      1.97
1         Panting     104        4.96      1.68        5.37      1.66
2           Puppy     105        2.88      2.14         6.4      2.13
3          Growl1     106        3.37      1.64        6.39      1.62
4             Dog     107        5.47      2.22        5.85      1.81
5        Carousel     109         6.4      2.13        5.64      1.84
6            Baby     110        7.64       2.1        6.03      1.98
7        MusicBox     111        6.01      2.19        5.65      1.91
8           Kids1     112        6.84      1.72        4.46      2.13
9            Cows     113        5.45      1.71        4.88      1.95
10         Cattle     114        5.01      1.85        6.04      1.81
11           Bees     115        2.16      1.33        7.03      1.91
12        Buzzing     116        3.02      1.65        6.51      2.13
13        Rooster   